In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
!pip install pycountry
import pycountry
import re
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
#Dash
!pip install dash-bootstrap-components
import dash_bootstrap_components as dbc
! pip install seaborn
import seaborn as sns
import dash_table
import plotly.graph_objects as go
!pip install wordcloud
from wordcloud import WordCloud
from collections import Counter
! pip install spacy
! python -m spacy download en_core_web_sm
! pip install --upgrade dash-bootstrap-components

df = pd.read_csv('master_with_sentiment_shifted_emotions_ner.csv', index_col=0)
topics = pd.read_csv('master_dataset_with_ner_topics.csv', index_col=0)[['id','topic_assigned']]
df.dropna(subset=['text'], inplace=True)
df.drop(columns=[col for col in df.columns if 'Unnamed' in col or '.1' in col], inplace=True)
df['clean_text'] = df['clean_text'].apply(lambda x: x.replace('gt','') if type(x) == str else x)
df = df.merge(topics, on='id', how='left')
# print(df.columns)

C:\Users\patri\AppData\Local\Temp\ipykernel_82836\2804423079.py:21: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


C:\Users\patri\AppData\Local\Temp\ipykernel_82836\2804423079.py:27: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('master_with_sentiment_shifted_emotions_ner.csv', index_col=0)
C:\Users\patri\AppData\Local\Temp\ipykernel_82836\2804423079.py:28: DtypeWarning: Columns (3,4,5,7,8,9,10,11,14,15,16,17,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  topics = pd.read_csv('master_dataset_with_ner_topics.csv', index_col=0)[['id','topic_assigned']]


In [2]:
def get_rid_of_quotes(text):
    if type(text) == str:
        text = text.split('\n')
        for sentence in text:
            if sentence == '':
                text.remove(sentence)
            if '&gt;' in sentence:
                text.remove(sentence)
        return '\n'.join(text)
df['post_id'] = df['id']
df['id'] = df.index
df['text'] = df['text'].apply(get_rid_of_quotes)

df.rename(columns={'TextBlob Sentiment':'TextBlob', 'Vader Sentiment_compound':'Vader','Flair Sentiment':'Flair'}, inplace=True)
df['flair'] = df['flair'].apply(lambda row: 'UNITED STATES' if row == 'USA' else row)
df['flair'] = df['flair'].apply(lambda row: 'UNITED KINGDOM' if row == 'UK' else row)
df['reply_to_flair'] = df['reply_to_flair'].apply(lambda row: 'UNITED STATES' if row == 'USA' else row)
df['reply_to_flair'] = df['reply_to_flair'].apply(lambda row: 'UNITED KINGDOM' if row == 'UK' else row)
og_df = df.copy()

bert1_cols = [cols for cols in df.columns if 'bert_' in cols]
bert2_cols = [cols for cols in df.columns if 'bert2_' in cols]

emotions_df = pd.concat([df[['date','flair']],df[bert1_cols]], axis=1)
emotions_df.columns = [col.split('_')[1] if 'bert' in col else col for col in emotions_df.columns]
emotions_df.head()

,date,flair,sadness,joy,love,anger,fear,surprise
0,04/06/2018,CZECHIA,0.004109,0.926028,0.002282,0.059248,0.006874,0.001459
1,07/06/2018,UNITED KINGDOM,0.002778,0.991360,0.000704,0.003555,0.001133,0.000471
2,19/06/2018,PORTUGAL,0.001437,0.985072,0.002855,0.008750,0.001398,0.000487
3,27/06/2018,POLAND,0.960252,0.007094,0.000550,0.028036,0.003517,0.000551
4,29/06/2018,DENMARK,0.002111,0.986545,0.001497,0.006760,0.002610,0.000477


In [3]:
### COUNTRIES MENTIONED (LEXICON-BASED)
!pip install unidecode
from unidecode import unidecode

df['flair']=df['flair'].apply(lambda row: unidecode(row))

df['reply_to_flair'] = df['reply_to_flair'].apply(lambda row: unidecode(row) if pd.notnull(row) else row)
countries = [country.name.lower() for country in pycountry.countries]
print(countries)
non_alphabet = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '
countries_with_non_alphabet = [country for country in countries if any(char in country for char in non_alphabet)]

countries = [unidecode(country) for country in countries]
countries = [f'british {country.split(",")[0]}' if ', british' in country else country for country in countries]
countries = [f'u.s. {country.split(",")[0]}' if ', u.s.' in country else country for country in countries]
countries = ['vietnam' if country == 'viet nam' else country for country in countries]
countries = [country.split(',')[0] for country in countries]
countries_multiple_names = { 'ivory coast': ['cote d\'ivoire','ivory coast' ], 'cocos (keeling) islands':['cocos islands', 'keeling islands'],
                'falkland islands': ['falkland islands', 'malvinas'], 'vatican city':['holy see', 'vatican city','vatican'],
                'laos':['laos', 'lao'], 'east timor':['east timor','timor-lest'], 'united arab emirates':['uae','united arab emirates'],
                'antigua and barbuda': ['antigua', 'barbuda'], 'bosnia and herzegovina':['bosnia', 'herzegovina'],
                'saint vincent and the grenadines':['saint vincent', 'grenadines','st vincent'], 'russia':['russia','russian federation'],
                'united states':['US', 'usa', 'america', 'murica','united states'], 'eswatini':['swaziland','eswatini'], 'sao tome and principe':['sao tome'],
                'united kingdom':['uk', 'england', 'scotland', 'wales', 'britain','united kingdom'], 'saint lucia':['st lucia']
                }
countries_with_non_alphabet

print(countries_multiple_names)
def check_country_in_text(text):
    text=text.lower()
    # print(text)
    countries_mentioned=[]
    for country in countries:
        if country in text:
            countries_mentioned.append(country.upper())
    for country, names in countries_multiple_names.items():
        for name in names:
            if name in text:
                countries_mentioned.append(country.upper())
        if country in text:
            countries_mentioned.append(country.upper())
    return countries_mentioned

df['country mentioned']  = df['text'].apply(lambda text: check_country_in_text(unidecode(text)))   

['aruba', 'afghanistan', 'angola', 'anguilla', 'åland islands', 'albania', 'andorra', 'united arab emirates', 'argentina', 'armenia', 'american samoa', 'antarctica', 'french southern territories', 'antigua and barbuda', 'australia', 'austria', 'azerbaijan', 'burundi', 'belgium', 'benin', 'bonaire, sint eustatius and saba', 'burkina faso', 'bangladesh', 'bulgaria', 'bahrain', 'bahamas', 'bosnia and herzegovina', 'saint barthélemy', 'belarus', 'belize', 'bermuda', 'bolivia, plurinational state of', 'brazil', 'barbados', 'brunei darussalam', 'bhutan', 'bouvet island', 'botswana', 'central african republic', 'canada', 'cocos (keeling) islands', 'switzerland', 'chile', 'china', "côte d'ivoire", 'cameroon', 'congo, the democratic republic of the', 'congo', 'cook islands', 'colombia', 'comoros', 'cabo verde', 'costa rica', 'cuba', 'curaçao', 'christmas island', 'cayman islands', 'cyprus', 'czechia', 'germany', 'djibouti', 'dominica', 'denmark', 'dominican republic', 'algeria', 'ecuador', 'egy

In [4]:
continent_df = pd.read_csv('continents.csv') 
continent_df.columns = ['flair', 'flair_continent']
continent_df['flair']=continent_df['flair'].apply(lambda row: row.upper()) 
continent_flair = pd.merge(df, continent_df, on='flair', how='left')


df.rename(columns={col: col.replace(' Sentiment', '').replace('_compound', '') for col in df.columns}, inplace=True)


df['Average Sentiment'] = df[['TextBlob', 'Vader', 'Flair', 'Flair Fast']].mean(axis=1)

continent_df.columns = ['reply_to_flair', 'reply_to_continent']
# print(df['country mentioned'])
continent_directed = pd.merge(df, continent_df, on='reply_to_flair', how='left')
# print(continent_directed[['reply_to_flair','reply_to_continent']])
# directed_emotions = emotions_df.explode('country mentioned')
# directed_emotions.dropna(subset=['country mentioned'], inplace=True)

emotions_df = pd.concat([df[['date','country mentioned']],df[bert1_cols]], axis=1)
df = df.explode('country mentioned')
continent_df.columns = ['country mentioned', 'mentioned_continent']

continent_mentioned = pd.merge(df, continent_df, on='country mentioned', how='left')

mentioned_emotions = emotions_df.explode('country mentioned')
mentioned_emotions.dropna(subset=['country mentioned'], inplace=True)

bert1_cols = [cols for cols in df.columns if 'bert_' in cols]
bert2_cols = [cols for cols in df.columns if 'bert2_' in cols]
for i in range(len(bert1_cols)):
    emotion = bert1_cols[i].split('_')[1]
    df[emotion] = df[[bert1_cols[i],[col for col in bert2_cols if bert1_cols[i].split('_')[1] in col][0]]].mean(axis=1)


In [5]:
### DIRECTED AND UNDIRECTED SENTIMENT
print(df.columns)
weights = [0.5, 0.1, 0.2, 0.3]  # Example weights for each column


average_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['Average Sentiment'].mean()
textblob_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['TextBlob'].mean()
vader_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['Vader'].mean()
flair_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['Flair'].mean()
flair_fast_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['Flair Fast'].mean()
flair_shift_sentiment = df.groupby(['country_code','flair','topic_assigned','clean_text','text', 'reply_to_flair','country mentioned'])['Flair Shifted'].mean()

aggregate_sentiment = pd.concat([average_sentiment, textblob_sentiment, vader_sentiment, flair_sentiment, flair_fast_sentiment, flair_shift_sentiment], axis=1)
aggregate_sentiment = pd.DataFrame(aggregate_sentiment).reset_index()

aggregate_sentiment.columns = ['Country Code','flair','topic_assigned','clean_text','text', 'reply_to_flair', 'Country Mentioned', 'Average Sentiment','TextBlob','Vader','Flair','Flair Shifted', 'Flair Fast']


undirected_sentiment = pd.DataFrame()
undirected_sentiment['Average Sentiment'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text',])['Average Sentiment'].mean()
undirected_sentiment['TextBlob'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text'])['TextBlob'].mean()
undirected_sentiment['Vader'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text'])['Vader'].mean()
undirected_sentiment['Flair'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text'])['Flair'].mean()
undirected_sentiment['Flair Fast'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text'])['Flair Fast'].mean()
undirected_sentiment['Flair Shifted'] = aggregate_sentiment.groupby(['flair','topic_assigned','clean_text','text'])['Flair Shifted'].mean()
undirected_sentiment.reset_index(drop=False, inplace=True)
undirected_sentiment.columns = ['country','topic_assigned','clean_text','text','Average Sentiment','TextBlob','Vader','Flair','Flair Fast','Flair Shifted']


directed_sentiment = pd.DataFrame()
directed_sentiment['Average Sentiment'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['Average Sentiment'].mean()
directed_sentiment['TextBlob'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['TextBlob'].mean()
directed_sentiment['Vader'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['Vader'].mean()
directed_sentiment['Flair'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['Flair'].mean()
directed_sentiment['Flair Fast'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['Flair Fast'].mean()
directed_sentiment['Flair Shifted'] = aggregate_sentiment.groupby(['reply_to_flair','topic_assigned','clean_text','text'])['Flair Shifted'].mean()
directed_sentiment.reset_index(drop=False, inplace=True)
directed_sentiment.columns = ['country','topic_assigned','clean_text','text','Average Sentiment','TextBlob','Vader','Flair','Flair Fast','Flair Shifted']

mentioned_sentiment = pd.DataFrame()
mentioned_sentiment['Average Sentiment'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['Average Sentiment'].mean()
mentioned_sentiment['TextBlob'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['TextBlob'].mean()
mentioned_sentiment['Vader'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['Vader'].mean()
mentioned_sentiment['Flair'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['Flair'].mean()
mentioned_sentiment['Flair Fast'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['Flair Fast'].mean()
mentioned_sentiment['Flair Shifted'] = aggregate_sentiment.groupby(['Country Mentioned','topic_assigned','clean_text','text'])['Flair Shifted'].mean()
mentioned_sentiment.reset_index(drop=False, inplace=True)
mentioned_sentiment.columns = ['country','topic_assigned','clean_text','text','Average Sentiment','TextBlob','Vader','Flair','Flair Fast','Flair Shifted']
undirected_sentiment['Weighted'] = (undirected_sentiment[['TextBlob','Vader','Flair', 'Flair Fast']] * weights).sum(axis=1) / sum(weights)
directed_sentiment['Weighted'] = (directed_sentiment[['TextBlob','Vader','Flair', 'Flair Fast']] * weights).sum(axis=1) / sum(weights)
mentioned_sentiment['Weighted'] = (mentioned_sentiment[['TextBlob','Vader','Flair', 'Flair Fast']] * weights).sum(axis=1) / sum(weights)

directed_sentiment.head()

Index(['id', 'text', 'timestamp', 'subreddit', 'upvotes', 'post_title',
       'flair', 'reply_to_id', 'replied_to_flair', 'date', 'year', 'month',
       'country_code', 'country_code_reply', 'clean_text', 'words', 'TextBlob',
       'Vader_neg', 'Vader_neu', 'Vader_pos', 'tokens', 'reply_to_flair',
       'Countries, cities, states',
       'Nationalities or religious or political groups', 'Any named language',
       'Flair', 'Flair Fast', 'bert2_anger', 'bert2_anticipation',
       'bert2_disgust', 'bert2_fear', 'bert2_joy', 'bert2_love',
       'bert2_optimism', 'bert2_pessimism', 'bert2_sadness', 'bert2_surprise',
       'bert2_trust', 'bert_sadness', 'bert_joy', 'bert_love', 'bert_anger',
       'bert_fear', 'bert_surprise', 'Flair Shifted', 'Vader',
       'topic_assigned', 'post_id', 'country mentioned', 'Average Sentiment',
       'sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
      dtype='object')


,country,topic_assigned,clean_text,text,Average Sentiment,TextBlob,Vader,Flair,Flair Fast,Flair Shifted,Weighted
0,ALBANIA,economy,Albania used NorthSouth division attitudes hom...,Albania used to have a North-South division on...,-0.144325,0.290000,0.7741,-0.9996,-0.99920,-0.6418,-0.252064
1,ALBANIA,economy,Go immigrations officers polite aside wont nee...,"Go, immigrations officers are very polite, asi...",0.606050,0.320000,0.7003,0.6382,0.27611,0.7657,0.400457
2,ALBANIA,economy,India China others grow every year India one p...,India is one of the poorest nations on the pla...,-0.282129,0.114583,0.5106,-0.9986,-0.99720,-0.7551,-0.355026
3,ALBANIA,economy,Sanctions nothing one nation completely delusi...,How can one nation be so completely delusional...,-0.398625,-0.040000,0.4329,-0.9996,-0.99920,-0.9878,-0.433082
4,ALBANIA,economy,Serbs saltiest creatures planet lmao would pai...,Serbs are the saltiest creatures on this plane...,-0.170150,0.500000,0.8176,-0.9997,-0.99940,-0.9985,-0.152727


In [6]:
import pycountry


def code_to_country_name(country_code):
    try:
        if len(country_code) == 3:
            country = pycountry.countries.get(alpha_3=country_code)
        else:
            country = pycountry.countries.get(alpha_2=country_code)

        if country:
            return country.name
        else:
            return "Country not found"
    except Exception as e:
        return str(e)

def country_name_to_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        if country:
            return country.alpha_3
        else:
            return "Country not found"
    except Exception as e:
        return str(e)

In [15]:
df['country'] = df['flair']
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")
df['year'] = df['date'].dt.year
df['comment length'] = df['text'].apply(lambda row: len(row))

def merge_columns(row):
    entity = []
    if pd.notnull(row['Countries, cities, states']):
        entity.append(row['Countries, cities, states'])
    if pd.notnull(row['Any named language']):
        entity.append(row['Any named language'])
    if pd.notnull(row['Nationalities or religious or political groups']):
        entity.append(row['Nationalities or religious or political groups'])
    if not entity:
        return ''
    return entity

df['entities'] = df.apply(merge_columns, axis=1)
df['entities'] = df['entities'].apply(lambda x: ', '.join(x))

table = df[df['Countries, cities, states'].notnull()][['flair','text','entities','upvotes','topic_assigned', 'Average Sentiment']].drop_duplicates()
table['Sentiment'] = table['Average Sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
table.columns = ['Commenter Country', 'Comment Text', 'Recognised Entities', 'Upvotes', 'Topics', 'Mean Sentiment', 'Sentiment']
table.drop(columns=['Mean Sentiment','Topics'], axis=1, inplace=True)
reddit_df = df.copy()

In [18]:
import dash
from dash import html, dcc, Input, Output, State
import pandas as pd
import plotly.express as px
import spacy
from spacy import displacy

emotion_colours = {'sadness':'#23B9D4', 'joy':'#F9CF34', 'anger':'#DB69A9', 'love': '#D42349', 'surprise':'#696BDB', 'fear':'#12A58A'}
emotions = ['sadness', 'joy', 'anger', 'love', 'surprise', 'fear']
model_colours = {'TextBlob':'#915eb8', 'Vader':'#fa77cc', 'Flair':'#5fbdd4', 'Flair Fast':'#22577a', 'Flair Fast':'#38a3a5', 'Weighted':'#8063a6'}
topics = ['rights', 'economy', 'politics', 'trade']
topics = df['topic_assigned'].unique()
topics = [topic for topic in topics if pd.notna(topic)]
topics.insert(0, 'All')

ner = spacy.load("en_core_web_sm")


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])

app.layout = html.Div([
    html.Div([
        html.H1("Sentiment by Country", id="section1", style={'display': 'inline'}),
        html.B(" based on Reddit data.", style={'display': 'inline'}),
        html.P("  Please be patient while plots load.", style={'display': 'inline'}),
        html.Div([
            html.A(' Go to sentiment over time', href='#section2', style={'display': 'inline','margin-right': '10px'}),
            html.A('Go to emotion over time', href='#section3', style={'display': 'inline','margin-right': '10px'}),
            html.A('Go to sentiment correlation animation', href='#section4', style={'display': 'inline','margin-right': '10px'}),
            html.A('Go to example of comments', href='#section5', style={'margin-right': '10px'}),
        ]),# style={'display': 'inline'},
    ]),
    dbc.Row([
        dbc.Col(
        html.P("  Map of aggregate sentiment by country. Select sentiment type and model. Click on a country to change the sentiment and emotion timeseries."),
        width=9),
        dbc.Col(html.Button('Reset after clicking on country', id='reset-clickdata', n_clicks=0), width=3),
    ]),
    html.Div([
        dbc.Row([
        dbc.Col(html.B("  Topic: "), width='auto'),
        dbc.Col(dcc.Dropdown(
                    id='topic-dropdown',
                    options=[{'label': topic.title(), 'value': topic} for topic in topics],
                    value=topics[0],  # Default value
                    clearable=False),
                    width=1,
        ),
        dbc.Col(html.B("Sentiment Type: "), width=1),
        dbc.Col(dcc.RadioItems(
            options=[
                {'label': ' Undirected ⬞', 'value': 'undirected'},
                {'label': ' Directed (mention)   ⬞', 'value': 'mentioned'},
                {'label': ' Directed (response)   ', 'value': 'directed'}
            ],
            value='undirected',
            id='radio-dataset',
            inline=True,
            style={'display': 'inline-block', 'margin-right': '30px'}),width='auto'),
        dbc.Col(dbc.Tooltip(
            [   html.Span("\u00A0 Select sentiment type:", style={'display': 'block'}),
                html.Span("\u00A0 Undirected: sentiment of people from a country", style={'display': 'block', 'font-style': 'italic'}),
                html.Span("\u00A0 Directed (mention): sentiment of country when it is talked about", style={'display': 'block', 'font-style': 'italic'}),
                html.Span("\u00A0 Directed (response): sentiment towards people from a country", style={'display': 'block', 'font-style': 'italic'})
            ],
            target="radio-dataset", style={'padding-top': '20px', 'text-align': 'center','display': 'inline'}), width='auto'
            ),#])]),
        dbc.Col(html.B("Sentiment Model: "), width=1),
        dbc.Col(dcc.Checklist(
                options=[{"label": " TextBlob    ⬞", "value": "TextBlob"},
                {"label": " Vader    ⬞", "value": "Vader"},
                {"label": " Flair    ⬞", "value": "Flair"},
                {"label": " Flair Fast   ", "value": "Flair Fast"},
                        ],
                value=["Vader"],id="checkbox-models-choropleth", inline=True),width='auto'),
                dbc.Tooltip("Select sentiment to display model (selecting multiple gives mean of all selected). Select all models for most accurate sentiment.",
            target="checkbox-models-choropleth", style={'display': 'block', 'font-style': 'italic'}),
            ])
        ]),
        dcc.Graph(id='choropleth'),
        html.H2('Sentiment Timeseries', id='section2'),
        
        html.Div([
            # html.Div([
                dbc.Row([
                dbc.Col(
                html.B("Select sentiment model to display:"), width=3),
                dbc.Col(dcc.Checklist(
                    options=[
                    {"label": " TextBlob    ⬞", "value": "TextBlob"},
                    {"label": " Vader    ⬞", "value": "Vader"},
                    {"label": " Flair    ⬞", "value": "Flair"},
                    {"label": " Flair Fast     ⬞", "value": "Flair Fast"},
                    # {"label": " Weighted ⬞", "value": "Weighted"},
                        {"label": "Average Sentiment", "value":"Average Sentiment"}
                    ],
                    value=["TextBlob","Vader","Flair", "Flair Fast"],
                    id="checkbox-models-timeseries",
                    inline=True  # Stack options vertically
                )
            , style={'margin-top': '20px'}, width=6)]),  
            dcc.Graph(id='timeseries')
        ]),
        html.Div([
        html.A('  Go to top', href='#section1', style={'margin-right': '10px'}),
        html.A('Go to emotion over time', href='#section3', style={'margin-right': '10px'}),
        html.A('Go to sentiment correlation animation', href='#section4', style={'margin-right': '10px'}),
        html.A('Go to example of comments', href='#section5', style={'margin-right': '10px'}),
    ]),
        html.H2(' Emotion Timeseries', id='section3'),
        html.Div([
            html.Div([
                dbc.Row([
                    dbc.Col(html.B("  Select emotions to display:", style={'display': 'inline'}), width=3),
                    dbc.Col(dcc.Checklist(
                        options=[
                            {"label": " Sadness  ⬞", "value": "sadness"},
                            {"label": " Joy  ⬞", "value": "joy"},
                            {"label": " Anger  ⬞", "value": "anger"},
                            {"label": " Fear  ⬞", "value": "fear"},
                            {"label": " Love  ⬞", "value": "love"},
                            {"label": " Surprise  ⬞", "value": "surprise"}],
                        value=['sadness', 'joy', 'love', 'anger', 'fear','surprise'],
                        id="checkbox-emotions-timeseries",
                        inline=True  
                ), width=8)
            ])#, style={'margin-top': '20px'}),  
            ]),
            dcc.Graph(id='emotions')
        ]),
        html.Div([
        html.A(' Go to top', href='#section1', style={'margin-right': '10px'}),
        html.A('Go to sentiment over time', href='#section2', style={'margin-right': '10px'}),
        html.A('Go to sentiment correlation animation', href='#section4', style={'margin-right': '10px'}),
        html.A('Go to example of comments', href='#section5', style={'margin-right': '10px'}),
    ]),
        html.H2(' Scatter Animation', id='section4'),
        html.P("   X-axis:",style={'display': 'inline-block'}),
        dcc.RadioItems(
            options=[{"label": " TextBlob    ⬞", "value": "TextBlob"},
                    {"label": " Vader    ⬞", "value": "Vader"},
                    {"label": " Flair    ⬞", "value": "Flair"},
                    {"label": " Flair Fast     ⬞", "value": "Flair Fast"}
                        ],
                value="TextBlob",
            id='radio-scatter-x',
            inline=True,
            style={'display': 'inline-block', 'margin-right': '30px'}),
        html.P("   Y-axis:",style={'display': 'inline-block'}),
        dcc.RadioItems(
            options=[{"label": " TextBlob    ⬞", "value": "TextBlob"},
                    {"label": " Vader    ⬞", "value": "Vader"},
                    {"label": " Flair    ⬞", "value": "Flair"},
                    {"label": " Flair Fast     ⬞", "value": "Flair Fast"}
                        ],
                value="Vader",
            id='radio-scatter-y',
            inline=True,
            style={'display': 'inline-block', 'margin-right': '30px'}),
        dcc.Graph(id='scatter_anim'),
        html.Div([
        html.A('Go to top', href='#section1', style={'margin-right': '10px'}),
        # html.A('Go to map', href='#section1', style={'margin-right': '10px'}),
        html.A('Go to sentiment over time', href='#section2', style={'margin-right': '10px'}),
        html.A('Go to emotion over time', href='#section3', style={'margin-right': '10px'}),
        # html.A('Go to sentiment correlation animation', href='#section4', style={'margin-right': '10px'}),
        html.A('Go to example of comments', href='#section5', style={'margin-right': '10px'}),
        
        dbc.Row([dbc.Col(html.H2(' Comments', id='section5'), width=3),
            dbc.Col(html.Button("Generate random set of comments", id="randomise-button")),
        ]),
        dbc.Row([
                dbc.Col(
                    dash_table.DataTable(id='table',##, striped=True,# bordered=True#, style_table={'overflowY': 'scroll'},
                    style_cell={'textAlign': 'left', 'whiteSpace': 'normal'},
                    editable=True,
                    # style_cell_conditional=[
                    style_data_conditional=[
            # },
                        {'if': {
                                'filter_query': '{Sentiment} = "Positive"',
                                'column_id': 'Sentiment'
                            }, 'backgroundColor': 'dodgerBlue', 'color': 'white'},
                        {'if': {
                                'filter_query': '{Sentiment} = "Negative"',
                                'column_id': 'Sentiment'
                            }, 'backgroundColor': 'red', 'color': 'white'},
                        ]
                        ), width=13,
                    ),
            ]),
        html.A('Go to top', href='#section1', style={'margin-right': '10px'}),
        html.A('Go to sentiment over time', href='#section2', style={'margin-right': '10px'}),
        html.A('Go to emotion over time', href='#section3', style={'margin-right': '10px'}),
        html.A('Go to sentiment correlation animation', href='#section4', style={'margin-right': '10px'}),

    ]),
    ])
# ])

# Define global variable
global prev_n_clicks
prev_n_clicks = 0

@app.callback(
    [Output('choropleth', 'figure'),
     Output('timeseries', 'figure'),
     Output('emotions', 'figure'),
     Output('scatter_anim', 'figure'),
    #  Output('wordcloud', 'figure'),
    #  Output('worddist', 'figure'),
     Output('table', 'data'),
     ],
    [Input('checkbox-models-choropleth', 'value'),
     Input('checkbox-models-timeseries', 'value'),
     Input('checkbox-emotions-timeseries', 'value'),
     Input('radio-dataset', 'value'),
     Input('choropleth','clickData'),
     Input('reset-clickdata', 'n_clicks'),
     Input('radio-scatter-y', 'value'),
     Input('radio-scatter-x', 'value'),
     Input('topic-dropdown', 'value'),
     Input('randomise-button', 'n_clicks')]
)
def update_graphs(selected_models_choropleth, selected_models_timeseries, selected_emotions, selected_dataset, clickData, n_clicks,\
                  scatter_y, scatter_x, topic, clicks):
    global prev_n_clicks  # Declare prev_n_clicks as global
    if n_clicks > prev_n_clicks:
        clickData = None
    prev_n_clicks = n_clicks

    country_name = ''
    # print(clickData)
    if clickData != None:
        country_name=clickData['points'][0]['location']
        undir_sentiment = reddit_df[reddit_df['flair']==country_name]
        dir_sentiment = reddit_df[reddit_df['country mentioned']==country_name]
        men_sentiment = reddit_df[reddit_df['reply_to_flair']==country_name]
        timeseries_df = reddit_df[reddit_df['flair']==country_name]
            
    else:
        timeseries_df = reddit_df

    if selected_dataset == 'directed' and country_name=='':
        df_selected = directed_sentiment
    elif selected_dataset == 'mentioned' and country_name=='':
        df_selected = mentioned_sentiment
    elif country_name =='':
        df_selected = undirected_sentiment
    elif selected_dataset == 'mentioned':
        df_selected = men_sentiment
    elif selected_dataset == 'directed':
        df_selected = dir_sentiment
    else:
        df_selected = undir_sentiment

    if topic != 'All':
        df_selected = df_selected[df_selected['topic_assigned']==topic]
    else:
        df_selected.drop(columns=['topic_assigned'])

    word_list = ' '.join(df_selected['clean_text']).split()
    word_freq = {}
    for word in word_list:
        word_freq[word] = word_freq.get(word, 0) + 1
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    # Generate the word cloud
    wordcloud = WordCloud(width=2000, height=2700,colormap='viridis', background_color='white').generate_from_frequencies(dict(sorted_word_freq[:50]))


    wordcloud_array = wordcloud.to_array()
    fig_wordcloud = go.Figure()
    fig_wordcloud.add_trace(go.Image(z=wordcloud_array, hoverinfo='none'))
    fig_wordcloud.update_layout(
            title=f'{country_name} Word Cloud',
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            margin=dict(t=0, b=0, l=0, r=0),  # Adjust margin to remove extra space around the image
        )
    
    words = [item[0] for item in sorted_word_freq[:8]]
    frequencies = [item[1] for item in sorted_word_freq[:8]]
    # Create Plotly figure
    fig_worddist = go.Figure()
    fig_worddist.add_trace(go.Bar(x=words, y=frequencies))
    fig_worddist.update_layout(title=f'{country_name} Word Distribution',
                      xaxis_title='Word',
                      yaxis_title='Frequency',
                      yaxis=dict(tickangle=-90),
                      margin=dict(t=0, b=0, l=0, r=0),)

    df_mean_sentiment = df_selected[selected_models_choropleth].mean(axis=1).to_frame(name='Mean_Sentiment')
    
    df_mean_sentiment = pd.concat([df_mean_sentiment, df_selected['country']], axis=1)
    df_mean_sentiment.rename(columns={"Country mentioned":"country","reply_to_flair":"country"},inplace=True)
    
    fig_choropleth = None 
    if country_name == '':
        fig_choropleth = px.choropleth(df_mean_sentiment, locations='country', locationmode='country names',
                                    color='Mean_Sentiment', #color_continuous_scale=colourscale, 
                                    color_continuous_scale='Viridis',# hover_data=['Mean Sentiment','TextBlob','Vader','Flair', 'Flair Fast'],
                                    labels={'Mean_Sentiment': 'Sentiment'})
        timeseries_df = reddit_df
        
        emotions_df = reddit_df[[col for col in df.columns if col in emotions or col == 'date' or col == 'flair']]
    elif selected_dataset == 'mentioned':
        timeseries_df = reddit_df[reddit_df['country mentioned']==country_name]
        emotions_df = directed_emotions[directed_emotions['country mentioned']==country_name]
    elif selected_dataset == 'directed':
        timeseries_df = reddit_df[reddit_df['reply_to_flair']==country_name]
        emotions_df = df[[col for col in df.columns if col in emotions or 'date'==col or 'reply_to_flair'==col]]
        emotions_df = emotions_df[emotions_df['reply_to_flair']==country_name]
    else:
        timeseries_df = reddit_df[reddit_df['flair']==country_name]
        emotions_df = df[[col for col in df.columns if col in emotions or 'date'==col or 'flair'==col]]
        emotions_df = emotions_df[emotions_df['flair']==country_name]

    if len(timeseries_df)>500:
        timeseries_3days = timeseries_df[['date','TextBlob','Vader','Flair','Flair Fast', ]].resample('2D', on='date').mean().reset_index()
    else:
        timeseries_3days = timeseries_df[['date','TextBlob','Vader','Flair','Flair Fast']]

    emotions_df['date'] = pd.to_datetime(emotions_df['date'])
    if len(emotions_df)>700:
        emotions_3days = emotions_df[['date','anger', 'joy','sadness','fear','surprise','love']].resample('2D', on='date').mean().reset_index()
    else:
        emotions_3days = emotions_df[['date','anger', 'joy','sadness','fear','surprise','love']]
    
    fig_timeseries = px.line(timeseries_3days, x='date', y=selected_models_timeseries, color_discrete_map={model: model_colours[model] for model in selected_models_timeseries})
    
    if selected_dataset == 'mentioned':
        description = 'Directed (mention) '
    elif selected_dataset == 'directed':
        description = 'Directed (response) '
    else:
        description =''
    fig_timeseries.update_layout(title=f'{country_name} {description}Sentiment over Time', xaxis_title='Date', yaxis_title='Sentiment Score', legend_title='Sentiment Model')

    fig_emotions = px.line(emotions_3days, x='date', y=selected_emotions, color_discrete_map={emotion: emotion_colours[emotion] for emotion in selected_emotions})
    fig_emotions.update_layout(title=f'{country_name} {description}Emotions over Time', xaxis_title='Date', yaxis_title='Score', legend_title='Emotion')

    if selected_dataset == 'mentioned':
        scatter_df = continent_mentioned
        cols1 = ['country mentioned','mentioned_continent','date','TextBlob','Vader','Flair','Flair Shifted','Flair Fast','upvotes']
        cols2 = ['country mentioned','mentioned_continent']
    elif selected_dataset == 'directed':
        scatter_df = continent_directed
        cols1 = ['reply_to_flair','reply_to_continent','date','TextBlob','Vader','Flair', 'Flair Shifted','Flair Fast','upvotes']
        cols2 = ['reply_to_flair','reply_to_continent']
    else:
        scatter_df = continent_flair
        cols1 = ['flair','flair_continent','date','TextBlob','Vader','Flair', 'Flair Shifted','Flair Fast','upvotes']
        cols2 = ['flair','flair_continent']

    scatter_df['date'] = pd.to_datetime(scatter_df['date'], format="%d/%m/%Y")
    if 'Flair Sentiment Fast' in scatter_df.columns:
        scatter_df.rename(columns={'Flair Sentiment Fast':'Flair Fast'},inplace=True)

    scatter_df = scatter_df[cols1].groupby(cols2).resample('2M', on='date').agg({'TextBlob': 'mean','Vader': 'mean', 'Flair': 'mean',\
                                                    'Flair Shifted': 'mean','Flair Fast': 'mean', 'upvotes': 'mean'}).reset_index()
    scatter_df['upvotes'].fillna(0, inplace=True)

    scatter_df.rename(columns={'country mentioned':'country','reply_to_flair':'country','flair':'country'}, inplace=True)
    scatter_df.rename(columns={col: 'continent' for col in scatter_df.columns if 'continent' in col.lower()}, inplace=True)

    fig_scatter = px.scatter(scatter_df, x=scatter_x, y=scatter_y,size='upvotes',
                animation_frame='date', animation_group='country',color='continent', 
                size_max=60, width=1300, height=500,
                hover_data = ['country','TextBlob','Vader','Flair', 'Flair Fast','upvotes'],
                labels={'country': 'Country','continent': 'Continent'},
                title=f'Correlation of {description}sentiment over time by country (grouped by 2 month intervals)'
                )

    fig_scatter.update_layout(hoverlabel=dict(font_size=16, font_family="Rockwell"))

    fig_scatter.add_annotation(xref='paper',yref='paper',x=1.05,y=-.3,text=f'Size of bubbles proportional to upvotes',showarrow= False)

    fig_scatter.update_traces(hovertemplate="<br>".join([ "Country: %{customdata[0]}",
            "Sentiment Models:\n   TextBlob: %{customdata[1]:.2f}\n   Vader: %{customdata[2]:.2f}\n   Flair: %{customdata[3]:.2f}\n   Upvotes: %{customdata[4]}"])
    )
    fig_scatter.update_layout(xaxis=dict(range=[-1,1]),yaxis=dict(range=[-1,1]))
    
    # if clicks is None:
    #     return dash.no_update
    df_shuffled = table.sample(frac=1)
    subset_df = df_shuffled.head(5)
    data = subset_df.to_dict('records')
    
    return fig_choropleth, fig_timeseries, fig_emotions, fig_scatter, data

if __name__ == '__main__':
    app.run_server(port=8052, debug=True, dev_tools_props_check=False)